In [1]:
!nvidia-smi

Fri Mar  1 16:00:27 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.129.03             Driver Version: 535.129.03   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
!git clone https://github.com/bigcode-project/bigcode-evaluation-harness.git
%cd bigcode-evaluation-harness

Cloning into 'bigcode-evaluation-harness'...
remote: Enumerating objects: 4101, done.
remote: Counting objects: 100% (1683/1683), done.
remote: Compressing objects: 100% (511/511), done.
remote: Total 4101 (delta 1337), reused 1344 (delta 1163), pack-reused 2418
Receiving objects: 100% (4101/4101), 814.85 KiB | 6.91 MiB/s, done.
Resolving deltas: 100% (2787/2787), done.
/kaggle/working/bigcode-evaluation-harness


In [3]:
!pip install -e .

Obtaining file:///kaggle/working/bigcode-evaluation-harness
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.2/51.2 kB 863.2 kB/s eta 0:00:00 0:00:01
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 5.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.4/173.4 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 38.5 MB/s eta 0:00:00:00:0100:01
  Created wheel for pyext: filename=pyext-0.7-py3-none-any.whl size=7222 sha256=89960210b335a711eb41af334e515bf276281c926a9c1263da59e9224f37eb7c
  Stored in directory: /root/.cache/pip/wheels/09/95/a9/f3f15c5e52dec7912c332ae503e82fd680e576bf336437f002
  Created wheel for toolwrapper: filename=toolwrapper-2.1.0-py3-none-any.whl size=3338 sha256=fb30b08cc0c2601c37ce87518a31be97d80b4c4a90be9

In [4]:
!ls

CONTRIBUTING.md      README.md		    finetuning	 requirements.txt
Dockerfile	     bigcode_eval	    leaderboard  setup.py
Dockerfile-multiple  bigcode_eval.egg-info  main.py	 templates
LICENSE		     docs		    makefile	 tests


In [5]:
!touch config.yaml

In [6]:
!ls

CONTRIBUTING.md      bigcode_eval	    leaderboard       templates
Dockerfile	     bigcode_eval.egg-info  main.py	      tests
Dockerfile-multiple  config.yaml	    makefile
LICENSE		     docs		    requirements.txt
README.md	     finetuning		    setup.py


In [7]:
%%writefile config.yaml
compute_environment: LOCAL_MACHINE
deepspeed_config: {}
distributed_type: MULTI_GPU
fsdp_config: {}
machine_rank: 0
main_process_ip: null
main_process_port: null
main_training_function: main
mixed_precision: no
num_machines: 1
num_processes: 1
gpu_ids: all
use_cpu: false

Overwriting config.yaml


In [8]:
my_file = open('config.yaml')

In [9]:
my_file.read()

'compute_environment: LOCAL_MACHINE\ndeepspeed_config: {}\ndistributed_type: MULTI_GPU\nfsdp_config: {}\nmachine_rank: 0\nmain_process_ip: null\nmain_process_port: null\nmain_training_function: main\nmixed_precision: no\nnum_machines: 1\nnum_processes: 1\ngpu_ids: all\nuse_cpu: false\n'

In [10]:
!git lfs install

Updated git hooks.
Git LFS initialized.


In [9]:
import torch
# setting GPU as default device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [11]:
device

device(type='cuda')

In [11]:
!huggingface-cli login --token <Token>

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [16]:
!accelerate launch --config_file config.yaml main.py \
  --model bigcode/starcoderbase-3b \
  --max_length_generation 512 \
  --tasks humaneval \
  --temperature 0.2 \
  --limit 10 \
  --n_samples 20 \
  --batch_size 20 \
  --use_auth_token \
  --allow_code_execution

2024-03-01 15:46:28.610659: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-01 15:46:28.610716: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-01 15:46:28.612174: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Selected Tasks: ['humaneval']
Loading model in fp32
/opt/conda/lib/python3.10/site-packages/transformers/models/auto/auto_factory.py:466: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
Loading checkpoint shards:   0%|                          | 0/2 [00:00<?, ?it/s]

In [12]:
!accelerate launch --config_file config.yaml main.py \
  --model codellama/CodeLlama-7b-hf \
  --max_length_generation 512 \
  --tasks humaneval \
  --temperature 0.2 \
  --limit 10 \
  --n_samples 20 \
  --batch_size 20 \
  --use_auth_token \
  --allow_code_execution

2024-03-01 16:10:17.465675: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-01 16:10:17.465801: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-01 16:10:17.636516: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Selected Tasks: ['humaneval']
Loading model in fp32
/opt/conda/lib/python3.10/site-packages/transformers/models/auto/auto_factory.py:466: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
config.json: 100%|█████████████████████████████| 637/637 [00:00<00:00, 2.25MB/s]

In [8]:
!accelerate launch  main.py \
  --model mlabonne/EvolCodeLlama-7b \
  --max_length_generation 200 \
  --tasks humaneval \
  --temperature 0.2 \
  --n_samples 50 \
  --batch_size 10 \
  --allow_code_execution

2024-02-29 21:25:49.790562: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-29 21:25:49.790562: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-29 21:25:49.790639: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-29 21:25:49.790701: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-29 21:25:50.073116: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory